In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class LSTMLyrics(nn.Module):
    def __init__(self, total_words, vector_size, word2vec_matrix, num_layers, units, features_size):
        super(LSTMLyrics, self).__init__()
        
        self.total_words = total_words
        self.vector_size = vector_size
        self.features_size = features_size
        self.num_layers = num_layers
        self.word2vec_matrix = word2vec_matrix
        self.units = units
        

        # The Embedding layer
        self.embedding = nn.Embedding(total_words, vector_size, padding_idx=0,
            _weight=torch.tensor(word2vec_matrix, dtype=torch.float))

        # LSTM
        self.b_rnn = nn.LSTM(vector_size + features_size, units, num_layers, batch_first=True)
        
        # Dense layer
        self.output_dense = nn.Linear(units, vector_size)
        
    
    def forward(self, lyrics_features_input, features_words, prev_state=None):
        
        if prev_state is None:
            # The first hidden state, usually a vecotr of zeros
            prev_state = self.init_hidden(lyrics_features_input)
        
        embedding_layer = self.embedding(lyrics_features_input)
        
        embedd_layer = torch.cat([embedding_layer, features_words], dim=-1)
        
        output, state = self.b_rnn(embedd_layer, prev_state)
        logits = self.output_dense(output)
        return logits

    
    def init_hidden(self, variable):
        # h is the last hidden state from each layer shape (batch, num layers, hidden_size)
        return (torch.zeros((self.num_layers, variable.size(0), self.units), device=variable.device),
                torch.zeros((self.num_layers, variable.size(0), self.units), device=variable.device))

In [2]:
class Custom_CosineEmbeddingLoss(nn.Module):
    def __init__(self):
        super(Custom_CosineEmbeddingLoss, self).__init__();
    
    def forward(self, predictions, targets):
        mat = np.zeros((predictions.shape[0],predictions.shape[1]))
        
        for index_batch, value_batch in enumerate(predictions):
            for index_seq, value_seq in enumerate(value_batch):
                target_seq = targets[index_batch, index_seq, :]
                mat[index_batch, index_seq] = 1- torch.dot(value_seq, target_seq) / (torch.norm(value_seq) * torch.norm(target_seq))
                
        return torch.tensor(np.mean(mat), dtype = torch.float, requires_grad=True).to(device)

In [ ]:
class Custom_L1_Loss(nn.Module):
    def __init__(self):
        super(Custom_L1_Loss, self).__init__();
        self.loss = nn.L1Loss(reduction='none')
    def forward(self, predictions, targets, tf):
        loss_value = self.loss(predictions,targets)
        mean_loss = loss_value.mean(dim=-1)
        weighted_loss = torch.mean(mean_loss/tf)
        return weighted_loss         
